# Python Scripts to Control the ASG Module

As it is well known, Python can be used to access the RedPitaya registers by using the *periphery module*. We will show how to  control the ASG module on the RedPitaya using only simple Python scripts.  This may not be a good approach when fast-response applications, however these scripts may help us to grasp some details of the operation of the RedPitaya board. 

## HW Configuration 

Connect IN1 to OUT1 on your RedPitaya board and the range jumper on IN1 to LV. If the *periphery* module is nor installed you may install it with the *!pip install periphery* command line on a new cell.

We will make extensive use the RedPitaya memory map, it  may be consulted here:

In [ ]:
from IPython.display import IFrame
IFrame("https://redpitaya.readthedocs.io/en/latest/developerGuide/software/build/fpga/regset_common.html", width=900, height=300)

## LED Blinking

If your board is connected the following script should blink the LEDs with the binary number 15 (00001111).

In [ ]:
#!/usr/bin/python
###################################inputs 
DATA = 15
###################################
from periphery import MMIO
import time
REGISTER_ADDRESS = 0x40000000 # Housekeeping
OFFSET = 0x30 # 0x30 LED control
ADDRESS_RANGE = 0x2 # LEDs 7-0 7:0 R/W
IP_BASE_ADDRESS = REGISTER_ADDRESS  + OFFSET
ADDRESS_OFFSET = 0
period = 1 # seconds
total = 5 # seconds
regset_gen = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
print("Blinking LED pattern %2d, i.e, %s, for %2d seconds" % (DATA, bin(DATA).replace("0b", ""), total))
for i in range(total):
    time.sleep(period/2.0)
    regset_gen.write16(ADDRESS_OFFSET,DATA)
    time.sleep(period/2.0)
    regset_gen.write16(ADDRESS_OFFSET,0)
regset.close()

## Read and Write to Registers

In general, to write data to the registers just pass the hex number of the register to the IP_BASE_ADDRESS variable and set the ADDRESS_RANGE and ADDRESS_OFFSET variables. For instance to write the 16-bit number 8 to the LED control register:

In [ ]:
# monitor.py
#!/usr/bin/python
###################################inputs 
IP_BASE_ADDRESS = 0x40000030 # LED control
#DATA = "" # read
DATA = 8 # write
###################################
from periphery import MMIO
import time
ADDRESS_RANGE = 0x2
ADDRESS_OFFSET = 0
regset_gen = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
if (DATA==""): # read
    val = regset_gen.read16(ADDRESS_OFFSET)
    print("Read: %2d" % (val))
else: # write
    regset_gen.write16(ADDRESS_OFFSET,DATA)
    print("Wrote: %2d" % (DATA))
regset.close()

To read the same register just comment out the line  DATA = 8 # write

In [ ]:
# monitor.py
#!/usr/bin/python
###################################inputs 
IP_BASE_ADDRESS = 0x40000030 # LED control
DATA = "" # read
#DATA = 8 # write
###################################
from periphery import MMIO
import time
ADDRESS_RANGE = 0x2
ADDRESS_OFFSET = 0
regset_gen = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
if (DATA==""): # read
    val = regset_gen.read16(ADDRESS_OFFSET)
    print("Read: %2d" % (val))
else: # write
    regset_gen.write16(ADDRESS_OFFSET,DATA)
    print("Wrote: %2d" % (DATA))
regset.close()


For instance to set the decimation value for the Oscilloscope module:


In [ ]:
#!/usr/bin/python
# set decimation
# Sample rate decimation is an integer in range [1, 2**17] = [1, 131072]
# Data decimation, supports only this values: 1,8, 64,1024,8192,65536. 
# If other value is written data will NOT be correct.
###################################inputs
from periphery import MMIO
DATA = 8 # write decimation value
###################################
REGISTER_ADDRESS = 0x40100000 # Oscilloscope
OFFSET = 0x14 # Data decimation
ADDRESS_RANGE = 0x4 # 16:0 R/W
IP_BASE_ADDRESS = REGISTER_ADDRESS  + OFFSET
ADDRESS_OFFSET = 0
print("IP_BASE_ADDRESS = ", hex(IP_BASE_ADDRESS))
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)

regset.write16(ADDRESS_OFFSET,DATA)
print("Wrote: ",DATA)
regset.close()

## Conversion from Volts to ADC Counts

To convert values in volts to ADC counts (in two´s complement format) we can use the following script.

In [ ]:
#!/usr/bin/python
###################################inputs
DATA_VOLT = .0472 # Volt
LV = True # +- 1 V
#LV = False # +- 20 V, Select one
###################################

import numpy as np

if (LV==True):
    if (DATA_VOLT > 1): DATA_VOLT = 1
    if (DATA_VOLT < -1): DATA_VOLT = -1
    if (DATA_VOLT>=0):
        DATA = int(np.round((DATA_VOLT)*2**13))-1
    else:
        DATA = int(np.round((1+DATA_VOLT)*2**13)+2**13) 
    
else:
    if (DATA_VOLT > 20): DATA_VOLT = 20
    if (DATA_VOLT < -20): DATA_VOLT = -20
    if (DATA_VOLT>=0):
        DATA = int(np.round((DATA_VOLT/20)*2**13))-1
    else: 
        DATA = int(np.round((1+DATA_VOLT/20)*2**13)+2**13) 
    

print("DATA_VOLT = %4f = %5d ADC COUNTS = %s = %s for LV = %s" % (DATA_VOLT,DATA,hex(DATA), bin(DATA).replace("0b", ""),bool(LV)))
print("If the value in volts is negative then ADC counts is greater than or equal to%5d " %(2**13))



## Conversion from ADC Counts to Volts 

To convert from ADC counts to volts we will use the code: 

In [ ]:
#!/usr/bin/python
###################################inputs
DATA = 15995 # ADC counts
LV = True # +- 1 V
HV = False # +- 20 V, Select one
###################################
if (DATA > 2**14-1): DATA = 2**14-1
if (DATA < 0): DATA = 0
if (LV==True): 
    if (DATA < 2**13):
        DATA_VOLT = (DATA+1)/2**13 
    if (DATA >= 2**13):
        DATA_VOLT =  (DATA-2**13)/2**13 -1
if (HV == True):
    if (DATA< 2**13):
        DATA_VOLT = 20*(DATA-1)/2**13 
    if (DATA >= 2**13):
        DATA_VOLT =  20*(DATA-2**13)/2**13 -1
    
print("If ADC counts is greater than or equal to%5d then the value in volts is negative" % (2**13))
print("%5d ADC COUNTS = %4f Volts for LV = %s" % (DATA,DATA_VOLT,bool(LV)))


## Setting a Trigger Threshold

Therefore to set the trigger threshold we can use:

In [ ]:
#!/usr/bin/python
###################################inputs
CHANNEL = 1 # 1 or 2
DATA = .5 # In Volts or ADC counts
Volt = True;
# Volt = False; Select one
LV = True # +- 1 V
#LV = False # +- 20 V, Select one
###################################
from periphery import MMIO
import numpy as np
if (Volt == True):
    DATA_VOLT = DATA
    if (LV==True):
        if (DATA_VOLT > 1): DATA_VOLT = 1
        if (DATA_VOLT < -1): DATA_VOLT = -1
        if (DATA_VOLT>=0):
            DATA = int(np.round((DATA_VOLT)*2**13))-1
        else:
            DATA = int(np.round((1+DATA_VOLT)*2**13)+2**13) 
    else:
        if (DATA_VOLT > 20): DATA_VOLT = 20
        if (DATA_VOLT < -20): DATA_VOLT = -20
        if (DATA_VOLT>=0):
            DATA = int(np.round((DATA_VOLT/20)*2**13))-1
        else: 
            DATA = int(np.round((1+DATA_VOLT/20)*2**13)+2**13) 
if (DATA > 2**14-1): DATA = 2**14-1
if (DATA < 0): DATA = 0
REGISTER_ADDRESS = 0x40100000 # Oscilloscope
OFFSET_A = 0x8 # Ch A threshold
OFFSET_B = 0xC # Ch B threshold
if (CHANNEL ==1) : OFFSET = OFFSET_A
elif (CHANNEL ==2) : OFFSET = OFFSET_B
ADDRESS_RANGE = 0x4 # 13:0 R/W
IP_BASE_ADDRESS = REGISTER_ADDRESS  + OFFSET
ADDRESS_OFFSET = 0
print("IP_BASE_ADDRESS = ", hex(IP_BASE_ADDRESS))
regset_gen = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
regset_gen.write16(ADDRESS_OFFSET,DATA)
print("Wrote: ",DATA)
val = regset_gen.read16(ADDRESS_OFFSET)
print("Read: %2d" % (val))
regset.close()

## Bash Commands

We can execute bash commands in jupyter cells. For instance, to get the current working directory on the RedPitaya¶


In [ ]:
import os 
cwd = os.getcwd() 
print("Current working directory:", cwd) 

To change directory

In [ ]:
import os 
os.chdir('../') 
cwd = os.getcwd() 
print("Current working directory:", cwd) 

To check the arguments that any particular command line accepts:¶

In [ ]:
!acquire

In [ ]:
!generate

## Generate a Waveform

To generate a sine wave with a frequency of .1 Mhz 

In [ ]:
#!/usr/bin/python
import subprocess
output = subprocess.check_output("generate 1 l 100000 sine ", shell=True)

or to generate an square wave with a frequency of 1 Mhz. Check that OUT1 is ON and BUST MODE for OUT1 is OFF on the scope app and adjust the trigger in NORMAL mode. Otherwise you may not see the wave. You should see something like ss_sqr.png

In [ ]:
!generate 1 l 1000000 sqr

## Acquire a Waveform 

To acquire and plot data we use the next script. Note that we have the option of calling "acquire .. -o" to get the values in volts or "acquire .. -x" to get the values in hexadecimal and we can do the conversion ourselves. The result should be the same. First let's acquire the data in volts.


Make sure the scope application is running on a separate window so you can see this notebook and the scope at the same time. On the scope make sure the Output1 generator is ON and the BURST MODE of Output1 is OFF. Also check that the trigger is in NORMAL mode. See Fig. ss_acq_1.png


In [ ]:
#!/usr/bin/python
import subprocess
import matplotlib.pyplot as plt
Hex= False
Volt = True # acquire 600 1 -o 
output = subprocess.check_output("acquire 600 1 -o ", shell=True)
output = output.decode("utf-8")
data_r=[]
if Volt == True:
    data_r=[float(j) for i,j in enumerate(output.split()) if i%2==0]
if Hex== True :
    for i,j in enumerate(output.split()):
        if i%2 == 0:
            ##data_r=[int((j[-4:]),16) % 2**14 for i,j in enumerate(output.split()) if i%2==0]
            DATA = int((j[-4:]),16) % 2**14
            if (DATA < 2**13):
                DATA_VOLT = (DATA+1)/2**13 
            if (DATA >= 2**13):
                DATA_VOLT =  (DATA-2**13)/2**13 -1
            data_r.append(DATA_VOLT)

#print (data_r)
fig = plt.figure(figsize=(10, 6))
plt.plot(data_r,'-ob')
plt.grid()
plt.show()

Now in Hex. The result should be the same. See Fig. ss_acq_2.png

In [ ]:
#!/usr/bin/python
import subprocess
import matplotlib.pyplot as plt
Hex= True # acquire 600 1 -x
Volt = False # acquire 600 1 -o 
output = subprocess.check_output("acquire 600 1 -x ", shell=True)
output = output.decode("utf-8")
data_r=[]
if Volt == True:
    data_r=[float(j) for i,j in enumerate(output.split()) if i%2==0]
if Hex== True :
    for i,j in enumerate(output.split()):
        if i%2 == 0:
            ##data_r=[int((j[-4:]),16) % 2**14 for i,j in enumerate(output.split()) if i%2==0]
            DATA = int((j[-4:]),16) % 2**14
            if (DATA < 2**13):
                DATA_VOLT = (DATA+1)/2**13 
            if (DATA >= 2**13):
                DATA_VOLT =  (DATA-2**13)/2**13 -1
            data_r.append(DATA_VOLT)

#print (data_r)
fig = plt.figure(figsize=(10, 6))
plt.plot(data_r,'-ob')
plt.grid()
plt.show()

## Sampling Rate

Note that the interval between consecutive markers is 8 nanosecond because the RedPitaya sampling rate is 125 MHz and 8 nanosecons is the inverse of 125 000 000 Hz.
 

If we acquire only 300 points we should see 2.4 cycles, i.e. 300*8 ns/(1 microsec). 

This time we will plot using Bokeh instread of Matplotlib. See Fig. ss_acq_3.png

In [ ]:
#!/usr/bin/python
from bokeh.resources import INLINE
import bokeh.io
from bokeh import *
bokeh.io.output_notebook(INLINE)
import subprocess
import matplotlib.pyplot as plt
import numpy as np

output = subprocess.check_output("acquire 300 1 -o ", shell=True)
output = output.decode("utf-8")
data_r=[float(j) for i,j in enumerate(output.split()) if i%2==0]

from bokeh.io import  show
from bokeh.plotting import figure
x = np.arange(np.size(data_r))
y = data_r
p = figure(width=800, height=400)
p.line(x, y, line_width=2)
#to use markers
p.circle(x, y, fill_color='green', size=4)

show(p)



# Arbitrary Signal Generator

Now we show how to control the Arbitrary Signal Generator. To do so we will use the RedPitaya to draw any function in various modes. The size of the data register id 16K, i.e., 1024*16 = 16,384. 

## The Elephant

Let's pretend, with a lot of imagination, that the following function represents an elephant. See Fig. ss_elephant.png.

In [ ]:
#!/usr/bin/python
from bokeh.resources import INLINE
import bokeh.io
from bokeh import *
bokeh.io.output_notebook(INLINE)
from bokeh.io import  show
from bokeh.plotting import figure
def elephant():
  Y=[]
  w = 2**14
  x1 = w/10
  x2 = w/2
  x3=w
  y1 =.8*x1/x1
  y21 = 1-np.exp (-x1/(w/5))
  y22 = 1-np.exp (-x2/(w/5))
  y32 = np.exp(-(x2-.7*w)**2/(w/6)**2)
  for x in range(w):
    if x<x1: y=.8*x/x1
    elif x<x2: y = 1-np.exp (-x/(w/5)) +(y1-y21)
    elif x<x3: 
        y = np.exp(-(x-.7*w)**2/(w/6)**2) -(y32-y22)+(y1-y21)
        if x>(w-w/7): y = (x/(w-w/7))*y
    Y.append(y-1)
  return Y



y = elephant()
x = np.arange(np.size(y))
p = figure(width=800, height=400)
p.line(x, y, line_width=2)
p.circle(x, y, fill_color='green', size=4)
show(p)



## Turn DAC1 OFF

Now we switch off DAC 1 by writing a 1 to bit 7 of the  0x0 Configuration offset of register 0x40200000 (remember that this address is used for the ASG module or check the memory map writeup.

In [ ]:
# monitor.py
#!/usr/bin/python
###################################inputs 
IP_BASE_ADDRESS = 0x40200000 # 0x0 Configuration
DATA = 0x80 # write
###################################
from periphery import MMIO
import time
ADDRESS_RANGE = 0x2
ADDRESS_OFFSET = 0
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
regset.write16(ADDRESS_OFFSET,DATA)
print("Wrote: %2d" % (DATA))
val = regset.read16(0)
print("Read: %2d" % (val))
regset.close()

Check on the scope app the Outpu1 stopped. You may change the trigger mode to AUTO to check that the square waveform disappears.

## Transfer an Arbitrary Waveform to the Memory Data Register

To transfer the previous  arbitrary function to the registers of the RedPitaya we use the following code.

- Note that we need to convert from volts to ADC values.

In [ ]:
#!/usr/bin/python
###################################inputs
CHANNEL = 1 # 1 or 2
Volt = True;
# Volt = False; Select one
LV = True # +- 1 V
#LV = False # +- 20 V, Select one
###################################
from periphery import MMIO
import numpy as np
REGISTER_ADDRESS = 0x40200000 # ASG



ADDRESS_RANGE = 0x2
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
regset.write16(0,0x80)   # DAC1 off

OFFSET = 0x10000 # Ch A memory data (16k samples)
ADDRESS_RANGE = 0x1FFFC - 0x10000 # 65532 = 16k * 4
IP_BASE_ADDRESS = REGISTER_ADDRESS  + OFFSET
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
V_ASG = elephant()
for adr in range(0,0x1FFFC-0x10000,0x4): # 65532
    i=int(adr/4)
    if (Volt == True):
        DATA_VOLT = V_ASG[i]
        if (LV==True):
            if (DATA_VOLT > 1): DATA_VOLT = 1
            if (DATA_VOLT < -1): DATA_VOLT = -1
            if (DATA_VOLT>=0):
                DATA = int(np.round((DATA_VOLT)*2**13))-1
            else:
                DATA = int(np.round((1+DATA_VOLT)*2**13)+2**13) 
        else:
            if (DATA_VOLT > 20): DATA_VOLT = 20
            if (DATA_VOLT < -20): DATA_VOLT = -20
            if (DATA_VOLT>=0):
                DATA = int(np.round((DATA_VOLT/20)*2**13))-1
            else: 
                DATA = int(np.round((1+DATA_VOLT/20)*2**13)+2**13) 
    if (DATA > 2**14-1): DATA = 2**14-1
    if (DATA < 0): DATA = 0
    regset.write16(adr,DATA)
regset.close()

Now we restart DAC1 by sending 1 to register 0x40200000 (check the memory map, 1-trig immediately)

In [ ]:
# monitor.py
#!/usr/bin/python
###################################inputs 
IP_BASE_ADDRESS = 0x40200000 # 0x0 Configuration
DATA = 1 # write
###################################
from periphery import MMIO
import time
ADDRESS_RANGE = 0x2
ADDRESS_OFFSET = 0
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
regset.write16(ADDRESS_OFFSET,DATA)
print("Wrote: %2d" % (DATA))
val = regset.read16(0)
print("Read: %2d" % (val))
regset.close()

You should see the elephants on the scope app in continuos mode.

## Setting the Waveform Frequency

Note that the frequency is the same 1 MHz as before. See Fig. ss_elephants.png

## The Ch A Counter Step Register

The following script reads the Ch A counter step register.

In [ ]:
# monitor.py
#!/usr/bin/python
###################################inputs 
IP_BASE_ADDRESS = 0x40200010 # Ch A counter step
###################################
from periphery import MMIO
ADDRESS_RANGE = 0x4
ADDRESS_OFFSET = 0
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
step = regset.read32(ADDRESS_OFFSET) % 2**30
print("Read: %2d" % (step))
regset.close()

The frequency,f, is related to the step value read above  according to the following equation:

$$ 
f = \frac{125 x10^6}{ BufferSize} * \frac{step} { 2^{16}}
$$

In [ ]:
BUFFER_SIZE=16384
step = 8589935
import numpy as np

f = (125000000 / BUFFER_SIZE) * (step / 2**16)
print("Frequency = ",np.round(f), "Hz")


Therefore to calculate the step for a given frequency use the following script, for instance for a frequency of .5 MHz

In [ ]:
f = 500000
BUFFER_SIZE=16384
step = int(2**16 * f /(125000000 / BUFFER_SIZE))
print ("Step =", step)

The step is half the value. 


Now we stop DAC1, write the new step value and restart DAC1 

In [ ]:
# monitor.py
#!/usr/bin/python
f = 500000 # Hz
BUFFER_SIZE=16384
step = int(2**16 * f /(125000000 / BUFFER_SIZE))
print ("Step =", step)
###################################inputs 
IP_BASE_ADDRESS = 0x40200000 # ASG base address
###################################
from periphery import MMIO
ADDRESS_RANGE = 0x14
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
regset.write16(0,0x80) # Stop DAC1

regset.write32(0x10,step) # write new step
regset.write16(0,1) # Start DAC1
regset.close()


You should see the waveform with the new frequency of .5 MHz (i.e., period of 2 microseconds). See Fig. ss_elephants_1.png.

## Burst Mode

Now we switch DAC1 to burst mode. 

According to the memory map writeup, we need to write to the following offset addresses


- 0x18 Ch A number of read cycles in one burst (Number of repeats of table readout. 0=infinite 15:0 R/W)


- 0x1C Ch A number of burst repetitions  (Number of repetitions. 0=disabled 15:0 R/W)

- 0x20 Ch A delay between burst repetitions (Delay between repetitions. Granularity=1us 31:0 R/W)


Let's generate 3 elephants, 1 us wide each,  with two replicas delayed 20 microseconds. 


In [4]:
# monitor.py
#!/usr/bin/python

###################################inputs 
IP_BASE_ADDRESS = 0x40200000 # ASG base address
###################################
from periphery import MMIO
ADDRESS_RANGE = 0x24
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
regset.write16(0,0x80) # Stop DAC1
regset.write16(0x18,3) # number of read cycles in one burst
regset.write16(0x1C,2) # number of burst repetitions 
regset.write32(0x20,20) # delay between burst repetitions 
regset.write16(0,1) # Start DAC1
regset.close()


We should see a bust of 3 elephants 1us wide each, followed by 2 replicas, each replica  delayed by 20 us. See Fig. ss_elephants_2.png

## Trigering

You can trigger burst mode manually by clicking the TRIG GEN button on Output1 of the scope app or by software with the following script 

In [13]:
# monitor.py
#!/usr/bin/python

###################################inputs 
IP_BASE_ADDRESS = 0x40200000 # ASG base address
###################################
from periphery import MMIO
ADDRESS_RANGE = 0x24
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
regset.write16(0,0x81) # Stop DAC1
regset.write16(0,0x80) # Change trigger mode
regset.write16(0,1) # Start DAC1 in 1-trig immediately configuration
regset.close()

## Returning to Continuous Mode

To return to continuous mode just write 0's to the burst registers.

In [14]:
# monitor.py
#!/usr/bin/python

###################################inputs 
IP_BASE_ADDRESS = 0x40200000 # ASG base address
###################################
from periphery import MMIO
ADDRESS_RANGE = 0x24
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
regset.write16(0,0x80) # Stop DAC1
regset.write16(0x18,0) # number of read cycles in one burst
regset.write16(0x1C,0) # number of burst repetitions 
regset.write16(0,1) # Start DAC1
regset.close()


In my system when I use the TRIG GEN button the first busts is distorted at its beginning. See Fig. ss_trig_manual.png. This problem does not occur if I start the bust by using the previous script. See Fig. ss_trig_soft.png.

# Conclusion

To put everything together let's suppose we wany to generate a burst consisting of the two exponential functions shown below, see Fig. ss_two_exp.png.


In [ ]:
#!/usr/bin/python
from bokeh.resources import INLINE
import bokeh.io
from bokeh import *
bokeh.io.output_notebook(INLINE)
from bokeh.io import  show
from bokeh.plotting import figure
import numpy as np
def wave():
    Y=[]
    w = 2**14
    tau = w/10
    x1 = w/2
    for x in range(w):
        if x<x1:y = np.exp(-x/tau) 
        else : y = .5*np.exp(-(x-x1)/tau) 
        Y.append(-y)  
    return Y
Y = wave()
X = np.arange(np.size(Y))
p = figure(width=800, height=400)
p.line(X, Y, line_width=2)
p.circle(X, Y, fill_color='green', size=4)
show(p)


with the following parameters:

- frequency : 1 MHz

- Two cycles per burst

- One replica delayed 10 us

- Repeated 10 times at a rate of 1 Hz

## Single Script to Generate a Waveform in Burst Mode

In [9]:
#!/usr/bin/python
######################################## inputs
CHANNEL = 1 # 1 
Volt = True;
# Volt = False; Select one
LV = True # +- 1 V
#LV = False # +- 20 V, Select one
f = 100000 # Hz
n_cycles = 2 # each cycle of 1 us
n_replicas = 1
delay = 30 # us after the end of the 2 cycles
total = 10 # times to repeat burst structure
period = 1 # seconds
########################################
from periphery import MMIO
import time
import numpy as np
IP_BASE_ADDRESS = 0x40200000 # ASG

######################################## # define wave

def wave():
    Y=[]
    w = 2**14
    tau = w/10
    x1 = w/2
    for x in range(w):
        if x<x1:y = np.exp(-x/tau) 
        else : y = .5*np.exp(-(x-x1)/tau) 
        Y.append(-y)  
    return Y
######################################## Stop DAC1
ADDRESS_RANGE = 0x24
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
regset.write16(0,0x80) # Stop DAC1
######################################## calculate step
BUFFER_SIZE=16384
step = int(2**16 * f /(125000000 / BUFFER_SIZE))
print ("Step =", step)
######################################## write step for appropiate frequency
regset.write32(0x10,step) # write new step
######################################## configure burst structure 
regset.write16(0x18,n_cycles) # number of read cycles in one burst
regset.write16(0x1C,n_replicas) # number of burst repetitions 
regset.write32(0x20,delay) # delay between burst repetitions in us
######################################## Copy wave to memory register
IP_BASE_ADDRESS += 0x10000 # Ch A memory data (16k samples)
ADDRESS_RANGE = 0x1FFFC - 0x10000 # 65532 = 16k * 4
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
V_ASG = wave()
for adr in range(0,0x1FFFC-0x10000,0x4): # 65532
    i=int(adr/4)
    if (Volt == True):
        DATA_VOLT = V_ASG[i]
        if (LV==True):
            if (DATA_VOLT > 1): DATA_VOLT = 1
            if (DATA_VOLT < -1): DATA_VOLT = -1
            if (DATA_VOLT>=0):
                DATA = int(np.round((DATA_VOLT)*2**13))-1
            else:
                DATA = int(np.round((1+DATA_VOLT)*2**13)+2**13) 
        else:
            if (DATA_VOLT > 20): DATA_VOLT = 20
            if (DATA_VOLT < -20): DATA_VOLT = -20
            if (DATA_VOLT>=0):
                DATA = int(np.round((DATA_VOLT/20)*2**13))-1
            else: 
                DATA = int(np.round((1+DATA_VOLT/20)*2**13)+2**13) 
    if (DATA > 2**14-1): DATA = 2**14-1
    if (DATA < 0): DATA = 0
    regset.write16(adr,DATA)
######################################## # start DAC1
IP_BASE_ADDRESS = 0x40200000 # ASG base address
ADDRESS_RANGE = 0x2
regset = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
######################################## number of triggers

for i in range(total):
    regset.write16(0,1) # Start DAC1 in 1-trig immediately configuration
    time.sleep(period)
    regset.write16(0,0)
regset.close()


Step = 858993


You should see something similar to ss_two_exp_scope.jpg

## The End